In [45]:
import os
import shutil
import codecs
import chardet
import json

def run_txt(input_dir, character):
    run = 1
    for file in os.listdir(input_dir):
        os.rename(f'{input_dir}{file}', f'{input_dir}{character}{run}.txt')
        run += 1

run_txt('./watcher/', 'watcher')

def run_tojson(input_dir, output_file):
    for file in os.listdir(input_dir):
        with open(f'{input_dir}{file}', 'r') as input_file:
            run_dict = input_file.readlines()
            with open(output_file, 'a') as output_json:
                json.dump(run_dict, output_json)

run_tojson('./watcher/', './watcher_runs.json')




